In [1]:
#Adding a lineup marker on our play-by-play data based on the data we have already collected

import pandas as pd

In [67]:
#This shouldn't (hopefully) be too hard. We have the starting lineups listed in the boxscore dataframe,
#  and every substitution throughout the game is an "enters" play in the play by play data

class lineups:
    def __init__(self, pbp):
        self.pbp = pbp
        self.cur_pbp_index = pbp.index[pbp["home_lineup_id"] == -1][0] # in case we have to run multiple times
        lineup_df = pd.read_csv("../data/lineups.csv")
        self.lineup_dict = {tuple([row.player_1, row.player_2, row.player_3, row.player_4, row.player_5]): row.lineup_id for _, row in lineup_df.iterrows()}
        #The above gives us an easily searchable dictionary to see if lineups are already represented.
        self.boxscore = pd.read_csv("../data/BoxScoreData.csv") #For starting lineups
        self.cur_home_lineup_id = -1
        self.cur_away_lineup_id = -1
        self.cur_game_id = -1


    def get_id(self, lineup):
        lineup = tuple(lineup)
        if lineup not in self.lineup_dict:
            try:
                to_df = {"lineup_id": [len(self.lineup_dict)], "player_1": [lineup[0]],
                        "player_2": [lineup[1]], "player_3": [lineup[2]],
                        "player_4": [lineup[3]], "player_5": [lineup[4]]}
                self.lineup_dict[lineup] = len(self.lineup_dict)
                to_df = pd.DataFrame.from_dict(to_df)
                to_df.to_csv("../data/lineups.csv", mode = 'a', header=False, index=False)
            except IndexError:
                print(lineup)
                print(self.home_lineup)
                print(self.away_lineup)
                print(self.cur_game_id)
        return self.lineup_dict[lineup]
        

    def switch_games(self, game_id):
        #We need to update the game_id and get starting lineups for both home and away teams
        self.cur_game_id = game_id
        players = self.boxscore.loc[(self.boxscore.Starter) & (self.boxscore.Game_ID == game_id)]
        home_players = sorted(players.loc[players.Home]["Player_ID"])
        away_players = sorted(players.loc[~players.Home]["Player_ID"])
        if (len(home_players) != 5) or (len(away_players) != 5):
            print(game_id) #probably gonna throw an error here, so I'm not even gonna bother stopping it
        
            
        home_id = self.get_id(home_players)
        away_id = self.get_id(away_players)

        print(game_id)
        self.home_lineup = home_players
        self.away_lineup = away_players

        return home_id, away_id
    

    def sub_player(self, subout, subin, team):
        if (team == "Home"):
            self.home_lineup.remove(subout)
            self.home_lineup.append(subin)
            self.home_lineup.sort()
            return self.get_id(self.home_lineup)

        if (team == "Away"):
            self.away_lineup.remove(subout)
            self.away_lineup.append(subin)
            self.away_lineup.sort()
            return self.get_id(self.away_lineup)
        

    def loop(self):
        df = self.pbp.loc[self.pbp.index >= self.cur_pbp_index]
        for _, row in df.iterrows():
            if row.game_id != self.cur_game_id:
                if (not row.game_id in self.boxscore.Game_ID.values):
                    print(f"skipped {row.game_id}")
                    continue

                self.cur_home_lineup_id, self.cur_away_lineup_id = self.switch_games(row.game_id)

            if row.play_type == "Enters":
                subout = row.secondary_player_id
                subin = row.player_id
                if (subout in self.home_lineup):
                    self.cur_home_lineup_id = self.sub_player(subout, subin, "Home")
                if (subout in self.away_lineup):
                    self.cur_away_lineup_id = self.sub_player(subout, subin, "Away")

            self.pbp.loc[self.cur_pbp_index, "home_lineup_id"] = self.cur_home_lineup_id
            self.pbp.loc[self.cur_pbp_index, "away_lineup_id"] = self.cur_away_lineup_id

            self.cur_pbp_index += 1
                

        


    

In [68]:
pbp = pd.read_csv("../data/FullPBPData.csv")
pbp = pbp.loc[pbp.game_id.isin(pd.read_csv("../data/BoxScoreData.csv").Game_ID.to_list())]


lin = lineups(pbp)
# #we don't have the boxscore data yet...
lin.loop()

10700
10701
10705
10706
10707
10709
10710
10711
10713
10714
10715
10716
10720
10721
10725
10729
10730
10733
10734
10735
10736
10737
10739
10740
10741
10744
10745
10746
10747
10749
10750
10751
10753
10754
10755
10758
10760
10761
10764
10765
10766
10767
10768
10771
10772
10776
10777
10779
10783
10784
10785
10786
10789
10790
10792
10793
10796
10797
10799
10800
10801
10804
10805
10806
10811
10812
10813
10815
10818
10819
10821
10824
10825
10827
10828
10829
10834
10835
10836
10837
10838
10841
10845
10846
10851
10853
10854
10855
10856
10857
10860
10862
10863
10865
10868
10871
10873
10874
10877
10878
10881
10884
10886
10887
10888
10890
10893
10895
10897
10899
10901
10905
10907
10909
10911
10913
10917
10919
10920
10921
10922
10923
10924
10926
10928
10931
10934
10936
10938
10939
10941
10943
10945
10951
10952
10953
10957
10959
10961
10965
10966
10968
10969
10970
10972
10973
10974
10975
10977
10979
10980
10981
10983
10985
10988
10989
10992
10993
10995
10998
10998
10999
11001
11003
11004
11005
1100

In [74]:
pbp2 = lin.pbp.drop(["Unnamed: 0"], axis =1)

pbp2.to_csv("../data/PlayByPlay.csv", index = False)

In [78]:
pbp2.loc[pbp2.away_lineup_id.isna()]

,game_id,time,quarter,player_id,play_type,secondary_player_id,away score,home score,rebound,shot_type,distance,shot_score,away_lineup_id,home_lineup_id
